# Buat Nama Kelompok DKK

## Import

In [26]:
#Ini gw copas dari tucil dulu ya sementara, nanti yang ga butuh apus ae

import numpy as np 
import random
import pandas as pd 
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
import itertools

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import tree
from sklearn.impute import SimpleImputer

from sklearn import svm, datasets
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, KFold

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

#Coba pake imputer


%matplotlib inline

## Read data

In [27]:
heartTrain = pd.read_csv('tubes2_HeartDisease_train.csv')
heartTest = pd.read_csv('tubes2_HeartDisease_test.csv')

## Analisis kondisi data

In [28]:
print('Tipe data pada kolom\n')
print(heartTrain.dtypes)
print()

print('Menampilkan sebagian data\n')
print(heartTrain.head())
print()

print('Mencari jumlah nilai yang tidak terdefinisi pada tiap kolom-nya')
heartTrain.isna().sum()

Tipe data pada kolom

Column1      int64
Column2      int64
Column3      int64
Column4     object
Column5     object
Column6     object
Column7     object
Column8     object
Column9     object
Column10    object
Column11    object
Column12    object
Column13    object
Column14     int64
dtype: object

Menampilkan sebagian data

   Column1  Column2  Column3 Column4 Column5 Column6 Column7 Column8 Column9  \
0       54        1        4     125     216       0       0     140       0   
1       55        1        4     158     217       0       0     110       1   
2       54        0        3     135     304       1       0     170       0   
3       48        0        3     120     195       0       0     125       0   
4       50        1        4     120       0       0       1     156       1   

  Column10 Column11 Column12 Column13  Column14  
0        0        ?        ?        ?         1  
1      2.5        2        ?        ?         1  
2        0        1        0        3  

Column1     0
Column2     0
Column3     0
Column4     0
Column5     0
Column6     0
Column7     1
Column8     0
Column9     0
Column10    0
Column11    0
Column12    0
Column13    0
Column14    0
dtype: int64

Berdasarkan dengan beberapa pengecekan di atas, dapat dilihat bahwa data pada csv yang diberikan:
1. Tidak semua data bertipe numerik
2. Ada beberapa data yang bernilai '?'
3. Ada data yang bernilai NaN (undefined)

Hal tersebut dapat mengganggu proses pemodelan. Oleh karena itu perlu dilakukan pre-processing sebagai berikut :

#### Konversi dataframe menjadi numerik

In [29]:
# Convert string to numeric, convert non-number to NAN
heartTrain = heartTrain.apply(pd.to_numeric,errors = 'coerce')

print('Tipe data pada kolom setelah konversi\n')
print(heartTrain.dtypes)
print()

# NaN count
print('Jumlah Nilai NaN setelah dataframe diconvert menjadi numerik\n')
print(heartTrain.isna().sum())

Tipe data pada kolom setelah konversi

Column1       int64
Column2       int64
Column3       int64
Column4     float64
Column5     float64
Column6     float64
Column7     float64
Column8     float64
Column9     float64
Column10    float64
Column11    float64
Column12    float64
Column13    float64
Column14      int64
dtype: object

Jumlah Nilai NaN setelah dataframe diconvert menjadi numerik

Column1       0
Column2       0
Column3       0
Column4      47
Column5      24
Column6      78
Column7       2
Column8      44
Column9      44
Column10     49
Column11    262
Column12    514
Column13    408
Column14      0
dtype: int64


#### Menghilangkan nilai NaN

Pada pre-processingnya, konversi data dari object (string) menjadi numerik berhasil menghilagkan tipe objek dari dataframe. Namun, untuk value yang tidak dapat dikonversi menjadi angka akan bernilai NaN yang membuat dataframe tidak bisa diolah. 

Salah satu cara termudah untuk menghilangkan nilai NaN adalah dengan cara menghapus row yang mengandung nilai tersebut, namun melihat pada column 12 terdapat 514 row yang bernilai NaN, cara ini tidak feasible karena akan sangat mengurangi data training. Oleh karena itum, kami memutuskan untuk me-replace nilai NaN dengan XXXXXX. Pemilihan XXXXXX dilakukan karena XXXXXXXXXXXXXXXXXXXXXXXXXXXX

In [32]:
imp = SimpleImputer(missing_values=np.nan, strategy='median')

c = heartTrain.columns
heartTrain = pd.DataFrame(imp.fit_transform(heartTrain))

heartTrain.columns = c
#imputed_DF.index = DF.index

#NaN count
print('Jumlah Nilai NaN setelah datafram di-imput\n')
print(heartTrain.isna().sum())

Jumlah Nilai NaN setelah datafram di-imput

Column1     0
Column2     0
Column3     0
Column4     0
Column5     0
Column6     0
Column7     0
Column8     0
Column9     0
Column10    0
Column11    0
Column12    0
Column13    0
Column14    0
dtype: int64


In [31]:
#Analisis buat milih algoritma(?)

In [7]:
#Split data train

In [ ]:
# Coba coba algoritma

In [ ]:
#Simpan model